In [2]:
import os
from IPython.display import Image as ipy_image
from IPython.display import display

#from utils.demo_util import get_shape_comp_model, get_shape_comp_opt, make_dummy_batch
#from utils.qual_util import load_bert2vqsc_model, get_lang_prob, save_mesh_as_gif
from utils.qual_util import get_lang_prob, save_mesh_as_gif
from utils.util import seed_everything
from utils.util_3d import init_mesh_renderer, sdf_to_mesh
import os
from termcolor import colored, cprint
import torch
import utils.util as util
from tqdm.notebook import tqdm
from datasets.text2shape import  Text2Shape


from datasets.text2shapepp_dataset import  Text2ShapePP


%reload_ext autoreload
%autoreload 2

In [74]:
class Opt():
    print("class")
    
def get_shape_comp_opt(gpu_id=0):
    opt = Opt()

    # args
    gpuid=[gpu_id]
    batch_size=16
    max_dataset_size=64

    name='new_bert_overfit'

    # default args
    opt.serial_batches = False
    opt.nThreads = 4

    # important args
    opt.dataset_mode = 'shapenet_code'
    opt.seed = 111
    opt.isTrain = True
    opt.gpu_ids = gpuid
    opt.max_dataset_size=64
    opt.device = 'cuda:%s' % gpuid[0]
    opt.batch_size = batch_size
    opt.max_dataset_size = max_dataset_size
    opt.logs_dir="logs"
    opt.name = name
    opt.lr = 1e-4
    opt.save_latest_freq = 1000
    opt.print_freq = 300
    #utils.util.seed_everything(opt.seed)
    opt.phase = 'test'
    opt.nepochs = 60
    opt.nepochs_decay = 60
    return opt
opt = get_shape_comp_opt()

class


In [75]:


dataset = Text2Shape()
#opt = Opt()

dataset.initialize(opt)

In [71]:
len(dataset)

64

In [48]:
for i in tqdm(range(len(dataset))):
    dataset[i]

  0%|          | 0/64 [00:00<?, ?it/s]

In [30]:
dataset[19]["current_text"]

'and rocking feet'

In [31]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

text1 = "thin legs"
text2 = "round arms"
# tokenized1 = tokenizer(text1,return_tensors='pt',padding=True)
# x = bertmodel(**tokenized1).pooler_output
# x.shape
# tokenized2 = tokenizer(text2,return_tensors='pt',padding=True)
# x2 = bertmodel(**tokenized2).pooler_output
# x2.shape
x1 = model.encode(text1)
x2 = model.encode(text2)
x1.shape
x2.shape
model.eval()



SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [147]:
cos = torch.nn.CosineSimilarity(dim=0)

In [148]:
cos(torch.Tensor(x1),torch.Tensor(x2))

tensor(0.4562)

In [18]:
from models.bert2vq_scmodel import BERT2VQSCModel

In [114]:
m = BERT2VQSCModel()
m.initialize(opt)

---------- Networks initialized -------------
-----------------------------------------------


In [122]:
# data = dataset[4]
# data2 = dataset[2]
# data["z_set_prev"] = data["z_set_prev"].unsqueeze(0)
# data["z_set_target"] = data["z_set_target"].unsqueeze(0)
# data["current_text"] = [data["current_text"]]


In [113]:
m.set_input(data)
m.forward()

NameError: name 'data' is not defined

In [125]:
m.backward()

In [10]:
x = dataset[0]["z_set_prev"].shape
y = dataset[3]["z_set_prev"].shape
x

torch.Size([8, 8, 8, 512])

In [11]:
x[x!=0].shape

AttributeError: 'int' object has no attribute 'shape'

In [85]:
from torch.utils.data import DataLoader
train_dl =  DataLoader(
            dataset,
            batch_size=opt.batch_size,
            shuffle=False,
            drop_last=True,
            num_workers=0)

In [115]:
options=opt
model = m
def train_one_epoch2(pt_profiler=None,epoch=0):
    global total_steps

    epoch_iter = 0
    for i, data in enumerate(train_dl):
        iter_start_time = time.time()
        #visualizer.reset()
        #import pdb;pdb.set_trace()
        total_steps += options.batch_size
        epoch_iter += options.batch_size
        model.set_input(data)

        model.optimize_parameters(total_steps)

        nBatches_has_trained = total_steps // options.batch_size
        # if total_steps % options.print_freq == 0:

        if nBatches_has_trained % options.print_freq == 0:
            errors = model.get_current_errors()
            #import pdb;pdb.set_trace()
            #print(model.loss,"LOSS")
            t = (time.time() - iter_start_time) / options.batch_size
#             visualizer.print_current_errors(
#                 epoch, epoch_iter, total_steps, errors, t)

        # if (nBatches_has_trained % options.display_freq == 0) or i == 0:
        #     # eval
        #     model.inference(data)
        #     visualizer.display_current_results(
        #        model.get_current_visuals(), total_steps, phase='train')

        #     #model.set_input(next(test_dg))
        #     test_data = next(test_dg)
        #     model.inference(test_data.unsqueeze(0))
        #     visualizer.display_current_results(
        #        model.get_current_visuals(), total_steps, phase='test')

#         if total_steps % options.save_latest_freq == 0:
#             cprint('saving the latestmodel (epoch %d, total_steps %d)' %
#                    (epoch, total_steps), 'blue')
#             latest_name = f'epoch-latest'
#             model.save(latest_name)

        if pt_profiler is not None:
            pt_profiler.step()

In [116]:
import time
cprint('[*] Start training. name: %s' % options.name, 'blue')
total_steps = 0
for epoch in tqdm(range(options.nepochs + options.nepochs_decay)):
    epoch_start_time = time.time()
    # epoch_iter = 0

    # profile
    train_one_epoch2(None,epoch)

#     if epoch % 100000 == 0:
#         cprint('saving the model at the end of epoch %d, iters %d' %
#                (epoch, total_steps), 'blue')
#         latest_name = f'epoch-latest'
#         model.save(latest_name)
#         cur_name = f'epoch-{epoch}'
#         model.save(cur_name)

    cprint(f'[*] End of epoch %d / %d \t Time Taken: %d sec \n%s' %
           (
               epoch, options.nepochs + options.nepochs_decay,
               time.time() - epoch_start_time,
               os.path.abspath(os.path.join(options.logs_dir, options.name))
           ), 'blue', attrs=['bold']
           )
    #import pdb;pdb.set_trace()
    print("Model Loss:", m.loss.item())
    model.update_learning_rate()


[*] Start training. name: new_bert_overfit


  0%|          | 0/120 [00:00<?, ?it/s]

[*] End of epoch 0 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 7659.30859375
[*] learning rate = 0.0000100
[*] End of epoch 1 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 7022.35791015625
[*] learning rate = 0.0000200
[*] End of epoch 2 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 5498.38427734375
[*] learning rate = 0.0000300
[*] End of epoch 3 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 4753.7177734375
[*] learning rate = 0.0000400
[*] End of epoch 4 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 4069.115234375
[*] learning rate = 0.0000500
[*] End of epoch 5 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/l

[*] End of epoch 46 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 806.1764526367188
[*] learning rate = 0.0000461
[*] End of epoch 47 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 762.822998046875
[*] learning rate = 0.0000456
[*] End of epoch 48 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 728.5869140625
[*] learning rate = 0.0000452
[*] End of epoch 49 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 680.9642944335938
[*] learning rate = 0.0000447
[*] End of epoch 50 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 724.18359375
[*] learning rate = 0.0000443
[*] End of epoch 51 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-a

[*] End of epoch 91 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 238.51254272460938
[*] learning rate = 0.0000330
[*] End of epoch 92 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 233.78326416015625
[*] learning rate = 0.0000328
[*] End of epoch 93 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 236.63449096679688
[*] learning rate = 0.0000326
[*] End of epoch 94 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 241.0787353515625
[*] learning rate = 0.0000324
[*] End of epoch 95 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 236.03012084960938
[*] learning rate = 0.0000323
[*] End of epoch 96 / 120 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Proj

In [117]:
x1 = dataset[0]
x2 = dataset[2]

In [91]:
x1.keys()

dict_keys(['current_text', 'z_set_prev', 'z_set_target'])

In [118]:
x1["current_text"] = [x1["current_text"]]
x1["z_set_prev"] = x1["z_set_prev"].unsqueeze(0)
x1["z_set_target"] = x1["z_set_target"].unsqueeze(0)
model.set_input(x1)

In [119]:
y = model.forward()

In [103]:
model.outp.shape

torch.Size([1, 8, 8, 8, 512])

In [123]:
k = x1["z_set_target"]

In [122]:
f = model.outp.cpu()

tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          ...,

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           

In [126]:
cos = torch.nn.CosineSimilarity(dim=-1)

In [127]:
cos(f,k)

tensor([[[[0.9999, 0.9958, 0.9955, 0.9957, 0.9959, 0.9958, 0.9954, 0.9997],
          [0.9999, 0.9952, 0.9952, 0.9955, 0.9957, 0.9947, 0.9946, 0.9973],
          [1.0000, 0.9953, 0.9948, 0.9942, 0.9949, 0.9942, 0.9942, 0.9968],
          [0.9999, 0.9943, 0.9937, 0.9857, 0.9774, 0.9824, 0.9852, 0.9829],
          [0.9999, 0.9951, 0.9952, 0.9788, 0.9809, 0.9690, 0.9852, 0.9926],
          [0.9999, 0.9926, 0.9735, 0.9795, 0.9678, 0.9693, 0.9838, 0.9916],
          [0.9999, 0.9956, 0.9857, 0.9838, 0.9956, 0.9957, 0.9956, 0.9961],
          [0.9995, 0.9991, 0.9933, 0.9942, 0.9961, 0.9974, 0.9999, 0.9997]],

         [[0.9999, 0.8901, 0.7856, 0.8462, 0.9661, 0.8152, 0.8811, 0.9990],
          [0.9996, 0.9501, 0.9036, 0.8073, 0.9610, 0.6888, 0.8082, 0.9614],
          [0.9998, 0.9826, 0.9550, 0.7903, 0.8607, 0.6336, 0.8461, 0.9948],
          [0.9997, 0.9533, 0.7750, 0.7222, 0.7655, 0.8609, 0.9249, 0.9894],
          [0.9999, 0.9663, 0.8292, 0.8587, 0.9248, 0.9210, 0.9274, 0.9794],
          